In [26]:
%use dataframe
%use kandy

In [27]:
val data = DataFrame.readCSV("C:\\Users\\Winggar\\Downloads\\phoible.csv")
data.columnNames()


[InventoryID, Glottocode, ISO6393, LanguageName, SpecificDialect, GlyphID, Phoneme, Allophones, Marginal, SegmentClass, Source, tone, stress, syllabic, short, long, consonantal, sonorant, continuant, delayedRelease, approximant, tap, trill, nasal, lateral, labial, round, labiodental, coronal, anterior, distributed, strident, dorsal, high, low, front, back, tense, retractedTongueRoot, advancedTongueRoot, periodicGlottalSource, epilaryngealSource, spreadGlottis, constrictedGlottis, fortis, raisedLarynxEjective, loweredLarynxImplosive, click]

In [28]:
enum class Signal { PLUS, MINUS, NONE }

fun (String).toSignal() = this.split(',').map {
    when (this) {
        "+" -> Signal.PLUS
        "-" -> Signal.MINUS
        else -> Signal.NONE
    }
}

fun (Char).toSignal() = when (this) {
    '+' -> Signal.PLUS
    '-' -> Signal.MINUS
    else -> Signal.NONE
}

data class Row(
    val inventoryId: Int,
    val glottocode: String?,
    val ISO6393: String?,
    val name: String?,
    val specificDialect: String?,

    val glyphId: String,

    val phoneme: String,
    val allophones: String?,
    val marginal: Boolean?,
    val segmentClass: String?,
    val source: String?,

    val tone: Signal,
    val stress: Signal,
    val syllabic: List<Signal>,
    val short: List<Signal>,
    val long: List<Signal>,
    val consonantal: List<Signal>,
    val sonorant: List<Signal>,
    val continuant: List<Signal>,
    val delayedRelease: List<Signal>,
    val approximant: List<Signal>,
    val tap: List<Signal>,
    val trill: List<Signal>,
    val nasal: List<Signal>,
    val lateral: List<Signal>,
    val labial: List<Signal>,
    val round: List<Signal>,
    val labiodental: List<Signal>,
    val coronal: List<Signal>,
    val anterior: List<Signal>,
    val distributed: List<Signal>,
    val strident: List<Signal>,
    val dorsal: List<Signal>,
    val high: List<Signal>,
    val low: List<Signal>,
    val front: List<Signal>,
    val back: List<Signal>,
    val tense: List<Signal>,
    val retractedTongueRoot: List<Signal>,
    val advancedTongueRoot: Signal,
    val periodicGlottalSource: List<Signal>,
    val epilaryngealSource: Signal,
    val spreadGlottis: List<Signal>,
    val constrictedGlottis: List<Signal>,
    val fortis: Signal,
    val raisedLarynxEjective: List<Signal>,
    val loweredLarynxImplosive: List<Signal>,
    val click: List<Signal>,
) {
    val isVowel get() = this.segmentClass == "vowel"
    val isConsonant get() = this.segmentClass == "consonant"
    val isTone get() = this.segmentClass == "tone"
}

val rows = data.map {
    Row(
        it.InventoryID,
        it.Glottocode,
        it.ISO6393,
        it.LanguageName,
        it.SpecificDialect,

        it.GlyphID.toString(),

        it.Phoneme,
        it.Allophones,
        it.Marginal,
        it.SegmentClass,
        it.Source,

        it.tone.toSignal(),
        it.stress.toSignal(),
        it.syllabic.toSignal(),
        it.short.toSignal(),
        it.long.toSignal(),
        it.consonantal.toSignal(),
        it.sonorant.toSignal(),
        it.continuant.toSignal(),
        it.delayedRelease.toSignal(),
        it.approximant.toSignal(),
        it.tap.toSignal(),
        it.trill.toSignal(),
        it.nasal.toSignal(),
        it.lateral.toSignal(),
        it.labial.toSignal(),
        it.round.toSignal(),
        it.labiodental.toSignal(),
        it.coronal.toSignal(),
        it.anterior.toSignal(),
        it.distributed.toSignal(),
        it.strident.toSignal(),
        it.dorsal.toSignal(),
        it.high.toSignal(),
        it.low.toSignal(),
        it.front.toSignal(),
        it.back.toSignal(),
        it.tense.toSignal(),
        it.retractedTongueRoot.toSignal(),
        it.advancedTongueRoot.toSignal(),
        it.periodicGlottalSource.toSignal(),
        it.epilaryngealSource.toSignal(),
        it.spreadGlottis.toSignal(),
        it.constrictedGlottis.toSignal(),
        it.fortis.toSignal(),
        it.raisedLarynxEjective.toSignal(),
        it.loweredLarynxImplosive.toSignal(),
        it.click.toSignal(),
    )
}
val languages = rows
    .groupBy { it.ISO6393 }
    .mapValues { (_, rows) ->
        rows
//        rows.filter { it.source == rows.first().source }
    }

fun languageName(iso: String) = languages[iso]!!.first().name


In [29]:
infix fun <T> (Collection<T>).has(element: T) = element in this

println(
    "%affricates: ${
        languages.filter { (_, rows) ->
            rows
                .filter { it.strident has Signal.PLUS && it.continuant has Signal.MINUS }
                .isNotEmpty()
        }.size / languages.size.toDouble()
    }"
)

println(
    "%plosives: ${
        languages.filter { (_, rows) ->
            rows.filter {
                it.strident has Signal.MINUS &&
                        it.continuant has Signal.MINUS &&
                        it.nasal has Signal.MINUS &&
                        it.click has Signal.MINUS &&
                        it.marginal == false &&
                        !it.phoneme.contains("|")
            }.isNotEmpty()
        }.size / languages.size.toDouble()
    }"
)

println(
    "%fricatives: ${
        languages.filter { (_, rows) ->
            rows.filter {

                it.continuant has Signal.PLUS &&
                        it.nasal has Signal.MINUS &&
                        it.click has Signal.MINUS &&
                        it.sonorant has Signal.MINUS &&
                        it.marginal == false &&
                        !it.phoneme.contains("|")
            }.isNotEmpty()
        }.size / languages.size.toDouble()
    }"
)

println(
    "%nasal: ${
        languages.filter { (_, rows) ->
            rows.filter {
                it.continuant has Signal.MINUS &&
                        it.nasal has Signal.PLUS &&
                        it.click has Signal.MINUS &&
                        it.sonorant has Signal.PLUS &&
                        it.marginal == false &&
                        !it.phoneme.contains("|")
            }.isNotEmpty()
        }.size / languages.size.toDouble()
    }"
)

println(
    "%semivowels: ${
        languages.filter { (_, rows) ->
            rows.filter {
                it.continuant has Signal.PLUS &&
                        it.nasal has Signal.MINUS &&
                        it.click has Signal.MINUS &&
                        it.sonorant has Signal.PLUS &&
                        it.approximant has Signal.PLUS &&
                        it.consonantal has Signal.MINUS &&
                        it.marginal == false &&
                        !it.phoneme.contains("|")
            }.isNotEmpty()
        }.size / languages.size.toDouble()
    }"
)

println(
    "%glides: ${
        languages.filter { (_, rows) ->
            rows.filter {
                it.continuant has Signal.PLUS &&
                        it.nasal has Signal.MINUS &&
                        it.click has Signal.MINUS &&
                        it.sonorant has Signal.PLUS &&
                        it.approximant has Signal.PLUS &&
                        it.consonantal has Signal.PLUS &&
                        it.marginal == false &&
                        !it.phoneme.contains("|")
            }.isNotEmpty()
        }.size / languages.size.toDouble()
    }"
)

%affricates: 0.6057142857142858
%plosives: 0.8104761904761905
%fricatives: 0.7219047619047619
%nasal: 0.840952380952381
%semivowels: 0.8128571428571428
%glides: 0.780952380952381


In [30]:
println(rows.filter { it.strident has Signal.PLUS && it.continuant has Signal.MINUS }.distinctBy { it.phoneme }.map { it.phoneme })

[t̠ʃ, t̠ʃʰ, t̠ʃˀ, t͈s, tsʰ, t͈sʷ, tsʷʰ, tsʷʼ, tsʼ, t̠͈ʃ, t̠͈ʃʷ, t̠ʃʷʰ, t̠ʃʷʼ, t̠ʃʼ, d̻z̻, t̻s̻, t̻s̻ʼ, ts, d̠ʒ, ɖʐ, ʈʂ, ʈʂʰ, d̠ʒ̤, d̠ʒː, t̠ʃː, dz, dzː, tsː, ˀd̠ʒ, n̠d̠ʒ, ndz, nts, d̠͈ʒ, tːs, ʈʂʼ, t̻s̻ʰ, t̠͈ʃː, t͉s, t̠ʃ͉, t̠ːʃ, tsʰː, tsʼː, t̠ʃʰː, t̠ʃʼː, ʰts, ɳɖʐ, ʰt̠ʃ, t̠ʃʲʰ, d̠ʒʷ, t̠ʃʷ, t̠ʃʲ, tsʲ, d̠ʒʲ, d̠ʒ̤ː, d̪z̪|dz, t̪s̪ʰ|tsʰ, t̪s̪|ts, t̪s̪ʷʰ|tsʷʰ, t̪s̪ʷʼ|tsʷʼ, t̪s̪ʼ|tsʼ, t̪s̪ʼː|tsʼː, t̪s̪ː|tsː, d̪z̪, t̪s̪ʰ, t̪s̪ʼ, n̠t̠ʃ, t̪s̪, t̪s̪ʲ, t̪s̪ˀ|tsˀ, n̪t̪s̪, n̪t̪s̪ʰ, n̠t̠ʃʰ, ɳʈʂ, ɳʈʂʰ, n̪t̪s̪|nts, tsɦ, tsˀ, dz̤, dzˠ, dzʼ, d̠ʒˠ, d̠ʒʼ, tsˠ, t̠ʃˠ, t̪s̪ʷː|tsʷː, t̠ʃʷː, t̪s̪ʲ|tsʲ, n̪d̪z̪, d̪z̪̤|dz̤, d̪z̪ʲ|dzʲ, t̪s̪ɦ|tsɦ, n̠d̠ʒʷ, dzʷ, ts̰, t̠ʃ̰, d̠ʒʱ, dzʱ, tsʷ, n̠̥t̠ʃ, t̺s̺, ts̪, t̰s̰, t̠ʃ̺ʰ, dzʲ, t̪s, ˀt̪s, dz̪, d̤z̤, d̤ʒ̤, n̠̩d̠ʒ, d̠̤ʒ̤, n̤d̤z̤, n̠̤d̠̤ʒ, ntsʼ, n̠t̠ʃʼ, n̠t̠ʃʷ, d̠ʒʰ, d̠ʒxʼ, t̠ʃx, t̪ʃ|t̠ʃ, ɖʐʷ, ʈʂʷ, d̠ʒɾ, n̠t̠ʃɾ, t̠ʃɾ, ndzʷ, d̤z, ndzʲ, tsʲʰ, d̪ʒ, t̪̻s̪̻, d̟z̟, d̟ʒ̟, t̟s̟, t̟ʃ̟, d̥z̥, d̤ʒ̤ː, d̻z̻ː, t̻s̻ː, d̠z̠ʲ, t̠s̠ʲʰ, d̪z̪ː, t̪s̪ː, ntsʰ, d̤z̤ː, d̠ːʒ, ʈʂː, t̠͉ʃ, ʰtsː, ʰt̠ʃː, tsˤ

In [31]:
val phonemeOccurrences = rows.map { it.phoneme }
    .toList()
    .flatMap { it.toList() }
    .distinct()
    .associateWith { symbol ->
        languages.filter { (_, rows) ->
            rows.filter { it.phoneme.contains(symbol) }.isNotEmpty()
        }.map { it.key }
    }

In [32]:
phonemeOccurrences
    .entries
    .map { it.key to it.value }
    .map { it.first to it.second.size }
    .sortedByDescending { it.second }
    .map {
        "${it.first} - ${it.second} - ${
            ({
                val ratio = it.second.toDouble() / languages.size.toDouble()
                "%.2f".format(ratio)
            })()
        }"
    }
    .forEach { println(it) }

t - 2093 - 1.00
k - 2083 - 0.99
i - 2055 - 0.98
m - 2046 - 0.97
n - 2045 - 0.97
a - 1992 - 0.95
p - 1990 - 0.95
u - 1971 - 0.94
j - 1925 - 0.92
w - 1816 - 0.86
l - 1734 - 0.83
s - 1676 - 0.80
e - 1647 - 0.78
o - 1617 - 0.77
b - 1466 - 0.70
d - 1463 - 0.70
ŋ - 1443 - 0.69
ɡ - 1346 - 0.64
h - 1254 - 0.60
r - 1175 - 0.56
ʃ - 1137 - 0.54
̠ - 1071 - 0.51
ɲ - 993 - 0.47
f - 968 - 0.46
ː - 905 - 0.43
ɛ - 896 - 0.43
ɔ - 869 - 0.41
z - 844 - 0.40
ʔ - 839 - 0.40
̪ - 791 - 0.38
ʒ - 786 - 0.37
ɾ - 635 - 0.30
v - 625 - 0.30
ə - 576 - 0.27
̃ - 517 - 0.25
˦ - 514 - 0.24
˨ - 505 - 0.24
c - 454 - 0.22
ʰ - 444 - 0.21
ʈ - 444 - 0.21
x - 433 - 0.21
ɨ - 425 - 0.20
ɟ - 393 - 0.19
ɪ - 382 - 0.18
ʷ - 380 - 0.18
̞ - 370 - 0.18
ʊ - 359 - 0.17
ɣ - 313 - 0.15
ɳ - 297 - 0.14
˧ - 286 - 0.14
ȵ - 276 - 0.13
ȶ - 276 - 0.13
β - 273 - 0.13
ɭ - 257 - 0.12
ɓ - 249 - 0.12
ɻ - 230 - 0.11
ɑ - 229 - 0.11
ʲ - 228 - 0.11
ɗ - 222 - 0.11
ɖ - 211 - 0.10
æ - 209 - 0.10
| - 209 - 0.10
q - 203 - 0.10
ç - 185 - 0.09
ʼ - 181 - 0.09
ɯ -

In [33]:
languages.filter { (key, _) -> key == "haw" }.values.first().first().name

Hawaiian

In [34]:
languages.entries.filter { (key, _) -> !phonemeOccurrences['t']!!.contains(key) }.map { (_, rows) -> rows.first().name }

[Hawaiian, CHEROKEE, JOMANG, BERTA, Abau, Talasa, Karajá]

In [35]:
fun chart(a: Char, b: Char) {
    println("$a: " + phonemeOccurrences[a]!!.size)
    println("$b: " + phonemeOccurrences[b]!!.size)
    println("$a ∧ $b: " + phonemeOccurrences[a]!!.intersect(phonemeOccurrences[b]!!).size)
    println("$a - $b: " + (phonemeOccurrences[a]!! - phonemeOccurrences[b]!!).size)
    println("$b - $a: " + (phonemeOccurrences[b]!! - phonemeOccurrences[a]!!).size)
    println("$a ⊕ $b: " + (phonemeOccurrences[a]!! - phonemeOccurrences[b]!!).union(phonemeOccurrences[b]!! - phonemeOccurrences[a]!!).size)
    println("all: " + languages.size)
}

chart('m', 'n')
chart('p', 'b')

val analyze =
    "m,n,ŋ,ɲ,p,t,k,b,d,ɡ,f,θ,ʃ,ʒ,v,z,h,j,w,ɹ,r,ɾ,l".split(',').map { it[0] }.filter { it in phonemeOccurrences }

val cooccurrenceData = analyze.map { symbolA ->
    analyze.map { symbolB ->
        val languagesWithA = phonemeOccurrences[symbolA]!!.toSet()
        val languagesWithB = phonemeOccurrences[symbolB]!!.toSet()

        val n = languages.size.toDouble()
        val n11 = languagesWithA.intersect(languagesWithB).size.toDouble()
        val n10 = (languagesWithA - languagesWithB).size.toDouble()
        val n01 = (languagesWithB - languagesWithA).size.toDouble()
        val n00 = n - n11 - n10 - n01

        val numerator = (n11 * n00) - (n10 * n01)
        val denominator = Math.sqrt((n11 + n10) * (n01 + n00) * (n11 + n01) * (n10 + n00))

        val result = if (denominator > 0) {
            numerator / denominator
        } else {
            0.0
        }

        result
    }
}

val cooccurrenceDF = dataFrameOf(
    "symbol" to analyze.map { it.toString() },
    *analyze.zip(cooccurrenceData).map { (symbol, values) ->
        symbol.toString() to values
    }.toTypedArray()
)

println("High correlation pairs (correlation > 0.3):")
val highCorrelationPairs = mutableListOf<Triple<Char, Char, Double>>()
for (i in analyze.indices) {
    for (j in (i + 1) until analyze.size) {
        val correlation = cooccurrenceData[i][j]
        if (correlation.absoluteValue > 0.3) {
            highCorrelationPairs.add(Triple(analyze[i], analyze[j], correlation))
        }
    }
}
highCorrelationPairs.sortedByDescending { it.third }.forEach { (a, b, correlation) ->
    println("$a - $b: ${"%.3f".format(correlation)}")
}

cooccurrenceDF


m: 2046
n: 2045
m ∧ n: 2034
m - n: 12
n - m: 11
m ⊕ n: 23
all: 2100
p: 1990
b: 1466
p ∧ b: 1370
p - b: 620
b - p: 96
p ⊕ b: 716
all: 2100
High correlation pairs (correlation > 0.3):
b - d: 0.886
d - ɡ: 0.815
b - ɡ: 0.801
m - n: 0.783
ʃ - ʒ: 0.615
f - v: 0.587
ɡ - z: 0.516
v - z: 0.514
d - z: 0.478
ɡ - ʒ: 0.468
d - ʒ: 0.463
b - f: 0.454
b - z: 0.452
d - f: 0.452
ɡ - f: 0.451
f - z: 0.450
ʒ - z: 0.418
b - ʒ: 0.416
j - w: 0.415
f - ʒ: 0.386
r - l: 0.361
ɡ - v: 0.355
d - v: 0.334
b - v: 0.321
ʒ - v: 0.321
ʃ - h: 0.314
ɲ - d: 0.314
ɲ - ɡ: 0.313
ɲ - b: 0.305
ŋ - l: 0.305
ɾ - l: -0.326
r - ɾ: -0.339


symbol,m,n,ŋ,ɲ,p,t,k,b,d,ɡ,f,θ,ʃ,ʒ,v,z,h,j,w,ɹ,r,ɾ,l
m,1.000000,0.783404,0.208322,0.123737,0.096839,0.042801,0.018897,-0.054411,-0.054843,-0.027522,0.071770,-0.011068,0.043696,0.051048,0.059691,0.041130,-0.059832,0.048983,-0.029056,0.002087,0.134621,-0.069901,0.242585
n,0.783404,1.000000,0.210889,0.131434,0.055127,0.042248,0.018460,-0.081868,-0.075782,-0.035725,0.043980,0.004535,0.034579,0.046739,0.048058,0.043209,-0.055670,0.080014,-0.012539,0.015510,0.124772,-0.054333,0.223338
ŋ,0.208322,0.210889,1.000000,0.225596,0.136384,0.032251,0.030733,0.010399,0.012757,0.079439,0.069734,-0.017779,-0.229368,0.004044,0.113518,-0.004082,-0.179405,0.052952,0.117572,0.111448,0.272266,-0.240017,0.304557
ɲ,0.123737,0.131434,0.225596,1.000000,0.017185,0.021677,0.000411,0.304972,0.313732,0.313212,0.277971,0.016130,0.216992,0.243078,0.186631,0.161289,0.073968,0.033647,0.031492,-0.085953,-0.022300,-0.025469,0.113306
p,0.096839,0.055127,0.136384,0.017185,1.000000,0.097649,0.074169,-0.089434,-0.094694,-0.069038,-0.052720,-0.057500,-0.010479,-0.038992,0.101621,-0.020884,-0.049305,0.037377,-0.011726,0.049541,0.058327,-0.017396,0.089181
t,0.042801,0.042248,0.032251,0.021677,0.097649,1.000000,0.271366,0.015956,-0.002217,0.008381,0.036905,-0.070369,0.013098,0.027656,0.037645,0.030556,-0.013812,0.042347,-0.022870,0.014425,0.031896,0.002099,-0.004791
k,0.018897,0.018460,0.030733,0.000411,0.074169,0.271366,1.000000,-0.024683,-0.036491,-0.012227,0.051558,0.018893,0.002179,0.003985,0.035561,0.019861,0.055814,0.049670,0.026432,0.022534,0.026888,0.013196,0.028537
b,-0.054411,-0.081868,0.010399,0.304972,-0.089434,0.015956,-0.024683,1.000000,0.886058,0.800807,0.454139,0.049529,0.233701,0.416452,0.321449,0.452340,0.267712,-0.018139,-0.050777,-0.176097,-0.033977,-0.025498,0.072466
d,-0.054843,-0.075782,0.012757,0.313732,-0.094694,-0.002217,-0.036491,0.886058,1.000000,0.814687,0.452250,0.034603,0.238845,0.463250,0.334371,0.477524,0.277474,-0.019052,-0.036793,-0.179498,-0.028343,-0.030185,0.084595
ɡ,-0.027522,-0.035725,0.079439,0.313212,-0.069038,0.008381,-0.012227,0.800807,0.814687,1.000000,0.451159,0.032678,0.237536,0.468101,0.354764,0.516353,0.192763,0.004190,-0.049257,-0.168401,0.049749,-0.058364,0.140223


In [36]:
val letter = 26
val offset = 15.0 / 26.0
val power = letter / 5.0
val eval = { x: Int -> (x + offset * letter).toDouble().pow(power) }
val sum = (1..letter).sumOf(eval)
(1..letter).forEach { println("%.2f".format(eval(it) / sum.toDouble() * 100)) }


0.11
0.14
0.19
0.26
0.34
0.43
0.55
0.70
0.87
1.07
1.32
1.60
1.94
2.32
2.77
3.29
3.88
4.55
5.31
6.18
7.15
8.25
9.48
10.85
12.37
14.07


In [37]:
import org.jetbrains.kotlinx.kandy.dsl.plot
import org.jetbrains.kotlinx.kandy.letsplot.layers.line
import org.jetbrains.kotlinx.kandy.letsplot.layers.points

val languagesFirstAnalysis = languages.mapValues { (_, rows) -> rows.filter { it.inventoryId == rows.first().inventoryId }}
val vowelCounts = languagesFirstAnalysis.mapValues { (_, rows) -> rows.count { it.segmentClass == "vowel" } }
val plots = vowelCounts.values.groupBy { it }.mapValues { (_, counts) -> counts.size }

println(vowelCounts.minBy { (_, count) -> count }.key)
println(vowelCounts.maxBy { (_, count) -> count }.key)

val vowelCountValues = plots.keys.toList()
val frequencyValues = plots.values.toList()

plot {
    line {
        x(vowelCountValues, name = "number of vowels")
        y(frequencyValues, name = "number of languages")
    }
    points {
        x(vowelCountValues)
        y(frequencyValues)
    }
}


gnd
thm


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="jBVG4O"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"number of vowels",
"y":"number of languages"
},
"stat":"identity",
"data":{
"number of vowels":[18.0,14.0,9.0,7.0,6.0,12.0,22.0,21.0,11.0,5.0,10.0,8.0,13.0,19.0,15.0,16.0,3.0,17.0,26.0,4.0,35.0,20.0,24.0,29.0,23.0,46.0,43.0,27.0,33.0,30.0,28.0,25.0,2.0,50.0,32.0,34.0,37.0,39.0,40.0,49.0],
"number of languages":[57.0,99.0,110.0,181.0,269.0,146.0,18.0,20.0,61.0,243.0,242.0,129.0,58.0,32.0,36.0,60.0,97.0,21.0,8.0,89.0,2.0,34.0,19.0,3.0,8.0,1.0,1.0,13.0,1.0,7.0,10.0,6.0,3.0,2.0,4.0,3.0,1.0,3.0,1.0,2.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"number of vowels"
},{
"type":"int",
"column":"number of languages"
}]
}
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[18.0,14.0,9.0,7.0,6.0,12.0,22.0,21.0,11.0,5.0,10.0,8.0,13.0,19.0,15.0,16.0,3.0,17.0,26.0,4.0,35.0,20.0,24.0,29.0,23.0,46.0,43.0,27.0,33.0,30.0,28.0,25.0,2.0,50.0,32.0,34.0,37.0,39.0,40.0,49.0],
"y":[57.0,99.0,110.0,181.0,269.0,146.0,18.0,20.0,61.0,243.0,242.0,129.0,58.0,32.0,36.0,60.0,97.0,21.0,8.0,89.0,2.0,34.0,19.0,3.0,8.0,1.0,1.0,13.0,1.0,7.0,10.0,6.0,3.0,2.0,4.0,3.0,1.0,3.0,1.0,2.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"x"
},{
"type":"int",
"column":"y"
}]
}
}],
"spec_id":"29"
};
 var containerDiv = document.getElementById("jBVG4O");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.275371983345245 323.1865671641791 L24.275371983345245 323.1865671641791 L34.3901103097391 214.455223880597 L44.50484863613295 223.7089552238806 L54.619586962526796 45.57462686567163 L64.73432528892066 15.499999999999943 L74.8490636153145 117.29104477611938 L84.96380194170835 177.44029850746267 L95.0785402681022 199.4179104477612 L105.19327859449605 46.73134328358208 L115.30801692088991 256.0970149253731 L125.42275524728376 157.77611940298505 L135.53749357367764 259.56716417910445 L145.65223190007148 212.14179104477608 L155.76697022646533 285.0149253731343 L165.88170855285918 257.25373134328356 L175.99644687925303 302.36567164179104 L186.11118520564688 260.7238805970149 L196.22592353204072 289.64179104477614 L206.34066185843457 287.3283582089552 L216.45540018482845 303.5223880597015 L226.5701385112223 305.8358208955224 L236.68487683761614 317.4029850746269 L246.79961516401 304.67910447761193 L256.9143534904038 319.7164179104478 L267.0290918167977 317.4029850746269 L277.1438301431915 311.6194029850746 L287.2585684695854 315.089552238806 L297.3733067959792 323.1865671641791 L307.4880451223731 318.55970149253733 L327.7175217751608 322.02985074626866 L337.83226010155465 325.5 L347.9469984279485 323.1865671641791 L358.06173675434235 324.34328358208955 L378.29121340713004 325.5 L398.52069005991774 323.1865671641791 L408.63542838631156 325.5 L438.97964336549313 325.5 L469.3238583446747 325.5 L499.6680733238563 324.34328

In [38]:
val consonantCounts = languagesFirstAnalysis.mapValues { (_, rows) -> rows.count { it.segmentClass == "consonant" } }
val plots = consonantCounts.values.groupBy { it }.mapValues { (_, counts) -> counts.size }

println(consonantCounts.minBy { (_, count) -> count }.key)
println(consonantCounts.maxBy { (_, count) -> count }.key)

val consonantCountValues = plots.keys.toList()
val frequencyValues = plots.values.toList()

plot {
    line {
        x(consonantCountValues, name = "number of consonants")
        y(frequencyValues, name = "number of languages")
    }
    points {
        x(consonantCountValues)
        y(frequencyValues)
    }
}


roo
nmn


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WVHIJh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"number of consonants",
"y":"number of languages"
},
"stat":"identity",
"data":{
"number of consonants":[22.0,18.0,60.0,49.0,29.0,38.0,46.0,47.0,23.0,32.0,31.0,19.0,39.0,21.0,27.0,35.0,16.0,34.0,26.0,24.0,30.0,33.0,10.0,9.0,17.0,20.0,13.0,14.0,11.0,15.0,8.0,36.0,37.0,25.0,44.0,42.0,45.0,28.0,52.0,58.0,74.0,50.0,71.0,12.0,81.0,54.0,57.0,95.0,6.0,59.0,43.0,56.0,40.0,41.0,77.0,55.0,48.0,62.0,61.0,75.0,51.0,130.0,53.0,63.0,79.0,64.0,99.0,113.0],
"number of languages":[123.0,100.0,1.0,4.0,50.0,27.0,9.0,10.0,99.0,25.0,39.0,127.0,15.0,103.0,61.0,26.0,84.0,32.0,70.0,109.0,49.0,33.0,17.0,9.0,136.0,145.0,65.0,78.0,44.0,73.0,4.0,24.0,25.0,70.0,10.0,8.0,9.0,51.0,2.0,3.0,2.0,3.0,1.0,56.0,1.0,3.0,1.0,2.0,1.0,4.0,6.0,3.0,14.0,6.0,1.0,1.0,4.0,1.0,2.0,1.0,6.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"number of consonants"
},{
"type":"int",
"column":"number of languages"
}]
}
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[22.0,18.0,60.0,49.0,29.0,38.0,46.0,47.0,23.0,32.0,31.0,19.0,39.0,21.0,27.0,35.0,16.0,34.0,26.0,24.0,30.0,33.0,10.0,9.0,17.0,20.0,13.0,14.0,11.0,15.0,8.0,36.0,37.0,25.0,44.0,42.0,45.0,28.0,52.0,58.0,74.0,50.0,71.0,12.0,81.0,54.0,57.0,95.0,6.0,59.0,43.0,56.0,40.0,41.0,77.0,55.0,48.0,62.0,61.0,75.0,51.0,130.0,53.0,63.0,79.0,64.0,99.0,113.0],
"y":[123.0,100.0,1.0,4.0,50.0,27.0,9.0,10.0,99.0,25.0,39.0,127.0,15.0,103.0,61.0,26.0,84.0,32.0,70.0,109.0,49.0,33.0,17.0,9.0,136.0,145.0,65.0,78.0,44.0,73.0,4.0,24.0,25.0,70.0,10.0,8.0,9.0,51.0,2.0,3.0,2.0,3.0,1.0,56.0,1.0,3.0,1.0,2.0,1.0,4.0,6.0,3.0,14.0,6.0,1.0,1.0,4.0,1.0,2.0,1.0,6.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"x"
},{
"type":"int",
"column":"y"
}]
}
}],
"spec_id":"32"
};
 var containerDiv = document.getElementById("WVHIJh");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.27537198334525 325.5 L24.27537198334525 325.5 L32.10613713926307 319.0416666666667 L36.02151971722198 308.27777777777777 L39.93690229518089 291.05555555555554 L43.8522848731398 232.93055555555554 L47.76766745109871 207.0972222222222 L51.683050029057625 187.7222222222222 L55.598432607016534 159.7361111111111 L59.51381518497545 170.49999999999997 L63.42919776293436 146.8194444444444 L67.34458034089327 34.87499999999994 L71.25996291885218 112.37499999999997 L75.1753454968111 54.24999999999994 L79.09072807477001 15.499999999999943 L83.00611065272892 105.91666666666663 L86.92149323068783 62.861111111111086 L90.83687580864674 114.52777777777774 L94.75225838660565 92.99999999999994 L98.66764096456457 176.95833333333331 L102.58302354252348 176.95833333333331 L106.49840612048239 196.33333333333331 L110.4137886984413 217.8611111111111 L114.3291712764002 220.0138888888888

In [39]:
languages.filter { (_, rows) -> rows.any { it.nasal has Signal.PLUS && it.segmentClass == "vowel" }}.size.toDouble() / languages.size

0.23857142857142857

In [45]:
val (Row).isMonophthong get() = this.isVowel && listOf(this.front, this.high, this.low, this.nasal, this.round).maxOf { it.size } == 1
val (Row).isDiphthong get() = this.isVowel && listOf(this.front, this.high, this.low, this.nasal, this.round).maxOf { it.size } == 2
val (Row).isTriphthong get() = this.isVowel && listOf(this.front, this.high, this.low, this.nasal, this.round).maxOf { it.size } == 3

val points =
    languagesFirstAnalysis.values
        .map { rows -> rows.count { it.isMonophthong } to rows.count { it.isDiphthong } }
        .groupBy { it }
        .mapValues { it.value.size }

println(languagesFirstAnalysis.maxBy { it.value.count { it.isDiphthong }}.value.first().ISO6393)
println(languagesFirstAnalysis.count { (_, rows) -> rows.any { it.isDiphthong }} / languagesFirstAnalysis.size.toDouble())

plot {
    points {
        x(points.keys.map { it.first }, name = "number of monophthongs")
        y(points.keys.map { it.second }, name = "number of diphthongs")
        color(points.values, name = "number of languages") {
            scale = continuous(Color.BLUE..Color.ORANGE, domain = 0..points.values.max())
        }
    }
}

dan
0.08904761904761904


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="t0jYnP"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"scale_mapper_kind":"color_gradient",
"high":"#fc8452",
"low":"#5470c6",
"limits":[0.0,268.0]
}],
"layers":[{
"mapping":{
"x":"number of monophthongs",
"y":"number of diphthongs",
"color":"number of languages"
},
"stat":"identity",
"data":{
"number of monophthongs":[18.0,14.0,9.0,7.0,6.0,12.0,22.0,21.0,11.0,5.0,10.0,18.0,8.0,11.0,19.0,19.0,15.0,16.0,3.0,17.0,26.0,16.0,13.0,4.0,35.0,10.0,20.0,9.0,24.0,6.0,29.0,20.0,15.0,9.0,7.0,7.0,5.0,5.0,18.0,6.0,7.0,5.0,15.0,12.0,12.0,5.0,9.0,24.0,12.0,18.0,9.0,16.0,5.0,27.0,23.0,33.0,30.0,28.0,25.0,2.0,9.0,18.0,22.0,12.0,18.0,6.0,16.0,4.0,7.0,18.0,8.0,10.0,18.0,44.0,9.0,5.0,5.0,6.0,8.0,5.0,10.0,20.0,17.0,4.0,12.0,20.0,19.0,8.0,10.0,16.0,12.0,8.0,13.0,10.0,10.0,20.0,13.0,11.0,10.0,5.0,37.0,14.0,39.0,22.0,9.0,17.0,14.0,14.0,40.0,5.0,19.0,8.0,34.0,6.0,17.0,21.0,10.0,6.0,16.0,11.0,13.0,11.0,20.0,9.0,9.0,11.0,15.0,8.0,18.0,12.0,13.0,18.0,11.0,24.0,19.0,16.0,21.0,32.0,8.0,11.0,13.0,14.0,11.0,15.0,37.0,9.0,13.0,18.0,15.0,23.0,31.0,7.0],
"number of diphthongs":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,3.0,3.0,1.0,2.0,3.0,2.0,3.0,6.0,4.0,5.0,6.0,1.0,6.0,8.0,7.0,8.0,22.0,3.0,25.0,3.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,3.0,9.0,2.0,15.0,3.0,4.0,4.0,4.0,2.0,7.0,17.0,6.0,11.0,5.0,9.0,3.0,4.0,15.0,2.0,12.0,7.0,8.0,10.0,6.0,15.0,8.0,6.0,7.0,1.0,6.0,6.0,4.0,3.0,4.0,1.0,4.0,14.0,20.0,0.0,8.0,0.0,4.0,5.0,6.0,4.0,5.0,0.0,1.0,8.0,1.0,0.0,2.0,32.0,8.0,11.0,6.0,4.0,16.0,17.0,1.0,1.0,9.0,6.0,3.0,11.0,11.0,9.0,4.0,7.0,8.0,8.0,4.0,14.0,6.0,11.0,16.0,18.0,5.0,8.0,2.0,6.0,4.0,2.0,12.0,11.0,12.0,2.0,9.0,18.0,8.0],
"number of languages":[47.0,86.0,99.0,177.0,268.0,133.0,10.0,12.0,51.0,243.0,231.0,3.0,119.0,3.0,1.0,23.0,27.0,50.0,97.0,16.0,4.0,2.0,47.0,89.0,1.0,4.0,29.0,3.0,13.0,1.0,2.0,1.0,4.0,1.0,2.0,3.0,3.0,6.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,5.0,1.0,4.0,1.0,2.0,9.0,4.0,1.0,5.0,9.0,3.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"number of monophthongs"
},{
"type":"int",
"column":"number of diphthongs"
},{
"type":"int",
"column":"number of languages"
}]
}
}],
"spec_id":"44"
};
 var containerDiv = document.getElementById("t0jYnP");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [46]:
import org.jetbrains.letsPlot.core.plot.base.Aes
import org.jetbrains.letsPlot.scale.guideLegend

val (Row).isBase get() = this.isMonophthong && this.nasal[0] == Signal.MINUS && this.long[0] == Signal.MINUS && this.short[0] == Signal.MINUS
val (Row).isModified get() = this.isMonophthong && !this.isBase

val points =
    languagesFirstAnalysis
        .mapValues { (_, rows) -> rows.count { it.isBase } to rows.count { it.isModified } }
        .entries
        .groupBy { it.value }
        .mapValues { (_, languages) -> languages.mapNotNull { languageName(it.key ?: return@mapNotNull null) } }

println(languagesFirstAnalysis.minBy { it.value.count { it.isBase } }.value.first().ISO6393)
println(languagesFirstAnalysis.maxBy { it.value.count { it.isBase } }.value.first().ISO6393)
println(languagesFirstAnalysis.minBy { it.value.count { it.isModified } }.value.first().ISO6393)
println(languagesFirstAnalysis.maxBy { it.value.count { it.isModified } }.value.first().ISO6393)

plot {
    withData(
        mapOf(
            "languages" to points.values.map { if (it.size > 10) it.take(10).plus("...") else it },
            "count" to points.values.map { it.size }
        )
    ) {
        points {
            x(points.keys.map { it.first }, name = "number of base segments")
            y(points.keys.map { it.second }, name = "number of modified segments")
            color(points.values.map { it.size }, name = "number of languages") {
                scale = continuous(Color.BLUE..Color.ORANGE, domain = 0..points.values.maxOf { it.size })
            }
            size(points.values.map { it.size }) {
                scale = continuous(1.5..15.0, 0..points.values.maxOf { it.size })
                legend { breaks(listOf()) }
            }
            tooltips("languages", "count")
        }
        abLine {
            slope.constant(1)
            intercept.constant(0)
        }
    }
}


kbd
kdj
unr
dib


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="0lkq6R"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"scale_mapper_kind":"color_gradient",
"high":"#fc8452",
"low":"#5470c6",
"limits":[0.0,254.0]
},{
"aesthetic":"size",
"breaks":[],
"range":[1.5,15.0],
"limits":[0.0,254.0]
}],
"layers":[{
"mapping":{
"x":"number of base segments",
"y":"number of modified segments",
"color":"number of languages",
"size":"size"
},
"stat":"identity",
"data":{
"number of base segments":[10.0,7.0,6.0,2.0,5.0,5.0,6.0,10.0,6.0,5.0,5.0,8.0,14.0,8.0,11.0,10.0,6.0,7.0,9.0,8.0,5.0,9.0,5.0,7.0,8.0,11.0,4.0,3.0,3.0,4.0,3.0,6.0,5.0,4.0,3.0,3.0,6.0,4.0,5.0,6.0,4.0,7.0,6.0,4.0,6.0,5.0,9.0,9.0,4.0,4.0,11.0,6.0,10.0,7.0,4.0,3.0,8.0,8.0,5.0,6.0,12.0,9.0,8.0,10.0,8.0,12.0,7.0,5.0,7.0,7.0,10.0,7.0,9.0,6.0,5.0,10.0,11.0,13.0,22.0,18.0,12.0,3.0,9.0,9.0,7.0,7.0,12.0,7.0,7.0,10.0,7.0,7.0,4.0,13.0,9.0,14.0,12.0,9.0,11.0,8.0,9.0,10.0,8.0,7.0,7.0,2.0,5.0,3.0,11.0,9.0,3.0,14.0,6.0,22.0,10.0,20.0,12.0,5.0,12.0,4.0,10.0,6.0,3.0,11.0,11.0,5.0,7.0,6.0,26.0,23.0,13.0,8.0,11.0,9.0,8.0,10.0,2.0,7.0,9.0,10.0,14.0,11.0,8.0,11.0,16.0,14.0,20.0,17.0,6.0,3.0,9.0,12.0,16.0,8.0,5.0,12.0,10.0,17.0,13.0,11.0,19.0,9.0,9.0,8.0,2.0,12.0,9.0,11.0,3.0,3.0,18.0,7.0,9.0,11.0,7.0,11.0],
"size":[9.0,63.0,7.0,2.0,9.0,7.0,4.0,1.0,24.0,254.0,171.0,3.0,5.0,4.0,2.0,1.0,159.0,5.0,83.0,5.0,2.0,30.0,15.0,159.0,71.0,1.0,15.0,105.0,2.0,5.0,7.0,7.0,6.0,2.0,96.0,2.0,1.0,34.0,14.0,3.0,4.0,2.0,81.0,1.0,1.0,3.0,6.0,12.0,77.0,1.0,1.0,15.0,22.0,43.0,7.0,14.0,8.0,31.0,6.0,3.0,1.0,1.0,1.0,8.0,15.0,1.0,2.0,6.0,1.0,2.0,3.0,3.0,8.0,6.0,2.0,49.0,15.0,8.0,2.0,3.0,2.0,1.0,7.0,8.0,2.0,16.0,3.0,7.0,6.0,6.0,9.0,8.0,4.0,2.0,8.0,1.0,2.0,1.0,1.0,11.0,1.0,4.0,4.0,5.0,1.0,3.0,2.0,4.0,5.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],
"languages":[["Korean","SOUTHERN NAMBIQUARA","sabaot","Semelai","Bambalang; Círàmbɔ́","Banjun","These","Kwanja","Võro"],["Ket","Kpelle","Wolof","Ewe","Khalkha","BAMBARA","NONI","EPENA PEDEE","avokaya","bozo","..."],["Lak","Wu","Kwakiutl","Ingush","Antiguan Creole","Besleri","Hmong Njua"],["Kabardian","Tigre"],["Georgian","Auyana","Zoque","Modern Hebrew","CHEROKEE","GWARI","Tangsa","Arabic","Tsez"],["Burushaski","Island Carib","Moseten","Belizean Creole","Orusyan","Ollari","Tay Daeng"],["Kurukh","Jemez","Tagwana","Galke"],["Telugu"],["Kota","Pomo","Luiseno","Ticuna","KANAKURU","MUMUYE","Cupeno","Bongo","Tadaksahak","Kannada","..."],["Mundari","Cantonese","Garo","Malagasy","Batak","Kunjen","Maranungku","Maung","Asmat","Kunimaipa","..."],["Kharia","Tagalog","Kaliai","Maori","Hawaiian","Wik-Munkan","Nasioi","Nootka","Nez Perce","Zuni","..."],["Khasi","Chuvash","Mauritian Creole"],["Sedang","Blang; Plang; Bulang; Pulang; Pula; Kawa; K''ala; Kontoi'","ngwe","Muyang","Ahi"],["Cambodian","akɔɔse","Balti","Nar-Phu"],["Vietnamese","Siwi"],["Mandarin Chinese"],["Hakka","Malay","Chamorro","Selepet","Maidu","Tarascan","Itonama","Araucanian","Margi","Kanuri","..."],["Yao","Yay","Sa'ban","Argobba","Pite Saami"],["Dafla","Lahu","Karen","Cham","Luo","Rumanian","Cheremis","Azerbaijani","KOTOKO","AIZI","..."],["Burmese","ALLADIAN","FUR","Petats","Northern Mansi"],["Lakkia","Kui"],["Thai","Maasai","KOLOKUMA IJO","KONKANI","abidji","bafut","ijo de l'est; Eastern Ijo","jur mödö","kroumen tépo","yaouré","..."],["Atayal","Nama","Yurak","M

In [48]:
val plosiveFricativeSets = languagesFirstAnalysis.mapValues { (_, rows) ->
    val plosivesFricatives = rows.filter {
        it.continuant has Signal.MINUS &&
                it.nasal has Signal.MINUS &&
                it.click has Signal.MINUS &&
                it.periodicGlottalSource.size == 1 &&
                it.spreadGlottis.size == 1
    }

    val classifications = plosivesFricatives.map { it.periodicGlottalSource[0] to it.spreadGlottis[0] }.distinct()
    classifications
}

val dataToDisplay = plosiveFricativeSets.values.groupingBy { it }.eachCount()
plot {
    pie {
        slice(dataToDisplay.values)
        fillColor(dataToDisplay.keys)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GCvmT9"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"slice":"slice",
"fill":"fill"
},
"stat":"identity",
"data":{
"slice":[77.0,1132.0,178.0,64.0,50.0,20.0,1.0,5.0,6.0,531.0,2.0,7.0,16.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0],
"fill":[[["MINUS","MINUS"],["MINUS","PLUS"]],[["PLUS","MINUS"],["MINUS","MINUS"]],[["PLUS","MINUS"],["MINUS","MINUS"],["MINUS","PLUS"]],[["PLUS","MINUS"],["MINUS","PLUS"],["MINUS","MINUS"]],[["PLUS","MINUS"],["PLUS","PLUS"],["MINUS","MINUS"],["MINUS","PLUS"]],[["PLUS","MINUS"],["MINUS","PLUS"]],[["PLUS","PLUS"],["MINUS","MINUS"],["MINUS","PLUS"],["PLUS","MINUS"]],[["MINUS","MINUS"],["MINUS","PLUS"],["PLUS","MINUS"]],[["MINUS","PLUS"],["MINUS","MINUS"]],[["MINUS","MINUS"]],[["MINUS","PLUS"]],[["PLUS","MINUS"]],[["MINUS","MINUS"],["PLUS","MINUS"]],[["PLUS","MINUS"],["MINUS","MINUS"],["PLUS","PLUS"],["MINUS","PLUS"]],[["MINUS","MINUS"],["PLUS","MINUS"],["MINUS","PLUS"]],[["PLUS","MINUS"],["PLUS","PLUS"],["MINUS","MINUS"]],[["PLUS","MINUS"],["MINUS","MINUS"],["MINUS","PLUS"],["PLUS","PLUS"]],[["PLUS","PLUS"],["PLUS","MINUS"],["MINUS","MINUS"],["MINUS","PLUS"]],[["PLUS","PLUS"],["PLUS","MINUS"],["MINUS","PLUS"],["MINUS","MINUS"]],[["PLUS","MINUS"],["PLUS","PLUS"],["MINUS","PLUS"],["MINUS","MINUS"]],[["MINUS","PLUS"],["PLUS","MINUS"]]]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"pie",
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"slice"
}]
}
}],
"spec_id":"50"
};
 var containerDiv = document.getElementById("GCvmT9");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 fill 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[MINUS, MINUS], [MINUS, 
 
 
 PLUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[PLUS, MINUS], [MINUS, 
 
 
 MINUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[PLUS, MINUS], [MINUS, 
 
 
 MINUS], [MINUS, PLUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[PLUS, MINUS], [MINUS, PLUS], 
 
 
 [MINUS, MINUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[PLUS, MINUS], [PLUS, PLUS], 
 
 
 [MINUS, MINUS], [MINUS, PLUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[PLUS, MINUS], [MINUS, PLUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[PLUS, PLUS], [MINUS, MINUS], 
 
 
 [MINUS, PLUS], [PLUS, MINUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[MINUS, MINUS], [MINUS, 
 
 
 PLUS], [PLUS, MINUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[MINUS, PLUS], [MINUS, 
 
 
 MINUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[MINUS, MINUS]] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 [[MINUS, PLUS]] 
 
 
